In [4]:
import numpy as np
import matplotlib.pyplot as plt 
import xarray as xr
import pyvista

In [23]:

#! wget https://storage.googleapis.com/ldeo-glaciology/elmer_janie/Surface.dat.names
#! wget https://storage.googleapis.com/ldeo-glaciology/elmer_janie/Results.dat.names


--2020-11-06 20:53:56--  https://storage.googleapis.com/ldeo-glaciology/elmer_janie/Results.dat.names
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.112.128, 74.125.124.128, 172.217.212.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.112.128|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2020-11-06 20:53:56 ERROR 404: Not Found.



In [20]:
surfurl = 'https://storage.googleapis.com/ldeo-glaciology/elmer_janie/output_vals/*.vtu'


In [13]:
type(surfdat)

numpy.ndarray

In [14]:
surfdat.shape

(243960, 24)